In [1]:
import pandas as pd
df=pd.read_csv('AQI_finaldataset.csv')
df = df.drop(['Location','CO','NO','NH3','O3'],axis=1)


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
# Load data

df=pd.read_csv('AQI_finaldataset.csv')
df = df.drop(['Location','CO','NO','NH3','O3'],axis=1)

# Filter data for a given location
location_id =0
df = df[df["Location_ID"] == location_id]

# Convert date column to datetime format
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

# Sort data by date
df.sort_values("Date", inplace=True)

# Extract target variables
targets = ["AQI", "NO2", "SO2", "PM10", "PM2.5"]
y = df[targets]

# Normalize data
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y)

# Define sequence length for LSTM
seq_length = 30

# Create input and output sequences
X, Y = [], []
for i in range(len(y_scaled) - seq_length):
    X.append(y_scaled[i:i+seq_length])
    Y.append(y_scaled[i+seq_length])
X = np.array(X)
Y = np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define model architecture
from keras.callbacks import EarlyStopping

# Define input layer
inputs = Input(shape=(seq_length, len(targets)))

# Define spatial graph convolution layer
for i in range (3):
    graph_conv = Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    graph_conv = MaxPooling1D(pool_size=2)(graph_conv)
    graph_conv = BatchNormalization()(graph_conv)

    # Define temporal graph convolution layer
    temp_conv = Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    temp_conv = MaxPooling1D(pool_size=2)(temp_conv)
    temp_conv = BatchNormalization()(temp_conv)

# Concatenate graph convolutions
    merged = Concatenate( axis=-1 )([graph_conv, temp_conv])

    # Define LSTM layer
lstm = LSTM(128)(merged)


# Define dense layers

dense = Dense(64, activation='relu')(lstm)
dense = Dropout(0.5)(dense)
dense = Dense(len(targets))(dense)
dense = Dense(64, activation='relu')(lstm)
dense = Dropout(0.5)(dense)
dense = Dense(len(targets))(dense)

# Define output layer
outputs = dense

# Define model
model = Model(inputs=inputs, outputs=outputs)

# Compile model
model.compile(loss='mse', optimizer='adam')

# Print model summary
# model.summary()

# Train model
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test), callbacks=[es], verbose=2)
# history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test), verbose=2)

# Make predictions on test data
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)

# Calculate root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: %.3f' % rmse)
print(y_pred[0])



Epoch 1/100
18/18 - 4s - loss: 0.1048 - val_loss: 0.0829 - 4s/epoch - 238ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0615 - val_loss: 0.0596 - 407ms/epoch - 23ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0498 - val_loss: 0.0602 - 387ms/epoch - 22ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0446 - val_loss: 0.0555 - 395ms/epoch - 22ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0387 - val_loss: 0.0514 - 475ms/epoch - 26ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0366 - val_loss: 0.0540 - 395ms/epoch - 22ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0340 - val_loss: 0.0539 - 393ms/epoch - 22ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0323 - val_loss: 0.0520 - 388ms/epoch - 22ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0291 - val_loss: 0.0526 - 384ms/epoch - 21ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0294 - val_loss: 0.0541 - 379ms/epoch - 21ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0292 - val_loss: 0.0530 - 391ms/epoch - 22ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0290 - val_loss: 0.0492 - 392ms/epoch - 22ms/step
Epo

In [ ]:
import pandas as pd
import numpy as np

# Define future date range
start_date = '2023-02-01'
end_date = '2023-02-28'
freq = 'D'
future_dates = pd.date_range(start=start_date, end=end_date, freq=freq)

# Create input sequences for future dates
last_seq = np.array(X_test[-1])  # last sequence from the test data
input_seq = last_seq.copy()
preds = []
for i in range(len(future_dates)):
    pred = model.predict(input_seq.reshape(1, seq_length, len(targets)))
    preds.append(pred)
    input_seq = np.append(input_seq[1:], pred, axis=0)

# Inverse transform the predicted values
preds = np.array(preds).reshape(-1, len(targets))
preds_inv = scaler.inverse_transform(preds)

# Convert predictions and dates to a pandas DataFrame
preds_df = pd.DataFrame(preds_inv, index=future_dates, columns=targets)



In [ ]:
preds_df